# Audit Intelligence System — Colab Step-by-Step (CrewAI-based)

This notebook will:
- Create CrewAI-style agents (Planner, Executor, Labeler, Reviewer, QA Generator, Fine-Tuner)
- Use SQLite via `pysqlite3-binary` for persistence
- Provide a hybrid labeling pipeline (rule-based + small LLM fallback)
- Index labeled data into FAISS for RAG
- Generate Q&A from labeled records and append to a single combined file
- Provide a basic fine-tuning stub (PEFT/LoRA-ready)
- Use `crewai` if available — otherwise fall back to lightweight agent stubs so you can run the pipeline



# Hugging face login for loading the authentication

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

# This will prompt for your Hugging Face token
login()


In [ ]:
from huggingface_hub import whoami

user_info = whoami()
print(user_info)


{'type': 'user', 'id': '651e544d977f6c20356de260', 'name': 'nivesh123', 'fullname': 'Nivesh Soni', 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/e08038814eabcb63bc82c8f8f1290054.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'model', 'role': 'fineGrained', 'createdAt': '2025-11-22T04:12:20.280Z', 'fineGrained': {'canReadGatedRepos': True, 'global': ['discussion.write', 'post.write'], 'scoped': [{'entity': {'_id': '651e544d977f6c20356de260', 'type': 'user', 'name': 'nivesh123'}, 'permissions': ['repo.content.read', 'repo.write', 'inference.serverless.write', 'inference.endpoints.infer.write', 'inference.endpoints.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write', 'discussion.write', 'user.billing.read', 'job.write']}]}}}}


In [ ]:
# Run this in a code cell (prefix with ! in notebook)
# Installs required packages. This may take a few minutes.
!pip install huggingface_hub
!pip install -q pysqlite3-binary sentence-transformers faiss-cpu transformers datasets peft accelerate streamlit
# Attempt to install crewai (if unavailable, it's okay; fallback will be used)
!pip install -q crewai || true

!pip install litellm
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.4 MB/s eta 0:0

## **Environment variables (dummy OpenAI key)**
We will set a dummy `OPENAI_API_KEY` (some libs expect it). In Colab, set it as an env var for this session.
This notebook uses a local model (`google/gemma-2b-it`) by default which does not require an OpenAI API key.


In [ ]:
# Set environment variables for the notebook session
import os
os.environ["OPENAI_API_KEY"] = "DUMMY_OPENAI_KEY"
os.environ["MODEL_NAME"] = "google/gemma-2b-it"
os.environ["EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"
os.environ["SQLITE_DB_PATH"] = "data/audit_ai.db"
os.environ["QA_COMBINED_PATH"] = "datasets/qa_data/combined_qa.json"
print("Environment variables set (in-memory).")


Environment variables set (in-memory).


## Project folders
Create the required folder structure.


In [ ]:
import os
os.makedirs("datasets", exist_ok=True)
os.makedirs("datasets/labeled_data", exist_ok=True)
os.makedirs("datasets/qa_data", exist_ok=True)
os.makedirs("outputs", exist_ok=True)
os.makedirs("outputs/meta", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("data", exist_ok=True)
print("Folders created.")


Folders created.


## Step: SQLite client & logger
We will use `pysqlite3-binary` as you requested. This cell creates a simple DB client and logger functions that other cells will call.


In [ ]:
# SQLite client (pysqlite3)
import pysqlite3 as sqlite3
import os, json
from datetime import datetime

DB_PATH = os.environ.get("SQLITE_DB_PATH", "data/audit_ai.db")
os.makedirs(os.path.dirname(DB_PATH) or ".", exist_ok=True)

_conn = None
def get_connection():
    global _conn
    if _conn is None:
        _conn = sqlite3.connect(DB_PATH, check_same_thread=False)
        _conn.row_factory = sqlite3.Row
        init_db(_conn)
    return _conn

def init_db(conn=None):
    if conn is None:
        conn = get_connection()
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT NOT NULL,
        agent TEXT NOT NULL,
        action TEXT NOT NULL,
        details TEXT
    )""")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS qa (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        question TEXT,
        answer TEXT,
        source_file TEXT,
        created_at TEXT
    )""")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS meta (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        doc_id TEXT,
        filename TEXT,
        metadata TEXT,
        created_at TEXT
    )""")
    conn.commit()

# initialize DB
get_connection()
print("SQLite DB initialized at", DB_PATH)


SQLite DB initialized at data/audit_ai.db


In [ ]:
# logger helper
from datetime import datetime
import json

def _now_iso():
    return datetime.utcnow().isoformat() + "Z"

def log_action(agent: str, action: str, details: dict | None = None):
    conn = get_connection()
    cur = conn.cursor()
    ts = _now_iso()
    details_json = json.dumps(details or {}, ensure_ascii=False)
    cur.execute(
        "INSERT INTO logs (timestamp, agent, action, details) VALUES (?, ?, ?, ?)",
        (ts, agent, action, details_json),
    )
    conn.commit()
    return cur.lastrowid

def fetch_recent_logs(limit: int = 10):
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT timestamp, agent, action, details FROM logs ORDER BY timestamp DESC LIMIT ?", (limit,))
    rows = cur.fetchall()
    out = []
    for r in rows:
        try:
            details = json.loads(r["details"] or "{}")
        except Exception:
            details = {"raw": r["details"]}
        out.append({"timestamp": r["timestamp"], "agent": r["agent"], "action": r["action"], "details": details})
    return out

print("Logger ready.")


Logger ready.


## CrewAI / agent framework
We will attempt to use `crewai.Agent` and `Crew` if installed. If not, a minimal local `Agent` stub will be used so the orchestration can still run in Colab.


In [ ]:
# Try to import crewai, otherwise define a minimal Agent/Crew stub
try:
    from crewai import Agent, Crew
    print("Using real crewai.Agent and Crew.")
except Exception as e:
    print("crewai not available or import failed — using lightweight Agent/Crew stubs.", e)
    class Agent:
        def __init__(self, role=None, goal=None, backstory=None, llm=None):
            self.role = role
            self.goal = goal
            self.backstory = backstory
            self.llm = llm
        def act(self, *args, **kwargs):
            return None
    class Crew:
        def __init__(self, agents=None, verbose=False):
            self.agents = agents or []
            self.verbose = verbose
        def run(self):
            return None


Using real crewai.Agent and Crew.


## Planner Agent
Creates a plan (simple deterministic) for the executor to run.


In [ ]:
import litellm
# agents/planner_agent.py equivalent
def plan_task(user_query: str):
    steps = [
        "Parse document",
        "Extract transactions",
        "Auto-label transactions",
        "Index labeled data into RAG",
        "Generate Q&A pairs",
        "Optionally fine-tune the model"
    ]
    log_action("Planner Agent", "Plan created", {"query": user_query, "steps": steps})
    return steps

planner_agent = Agent(
    role="Planner Agent",
    goal="Break user queries into sequential audit subtasks.",
    backstory="Planner that decomposes audit workflows.",
    llm=None
)
print("Planner agent ready.")


Planner agent ready.


## Executor Agent
Will coordinate parsing & calling the labeling / indexing / qa generation steps.


In [ ]:
# agents/executor_agent.py equivalent
import pandas as pd
from pathlib import Path

def execute_audit(file_path: str):
    # 1) parse (we assume CSV)
    df = pd.read_csv(file_path)
    # store original
    orig_path = os.path.join("datasets", "datasets_original_" + Path(file_path).name)
    df.to_csv(orig_path, index=False)
    log_action("Executor Agent", "Parsed document", {"file": file_path, "rows": len(df)})

    # 2) label
    labeled_csv = label_bank_statement(file_path)

    # 3) index into RAG
    indexed = index_labeled_file(labeled_csv)

    # 4) generate QA and append
    qa_path = generate_qa_from_labeled_data(labeled_csv)

    return labeled_csv, indexed, qa_path

executor_agent = Agent(role="Executor Agent", goal="Run audit tasks", backstory="Executor", llm=None)
print("Executor agent ready.")


Executor agent ready.


## Reviewer Agent
A simple rule-based reviewer to estimate labeling quality and recommend fine-tuning if needed.


In [ ]:
# agents/reviewer_agent.py equivalent
import pandas as pd

def simple_review_check(file_path: str):
    try:
        df = pd.read_csv(file_path)
        total = len(df)
        if total == 0:
            return {"accuracy": 0.0, "comments": "Empty dataset."}
        missing = df["CATEGORY"].isna().sum() if "CATEGORY" in df.columns else total
        missing_ratio = missing / total
        accuracy = round(max(0.0, 1.0 - missing_ratio - 0.05), 2)
        comments = []
        if missing_ratio > 0.1:
            comments.append("Too many unlabeled transactions.")
        if "CATEGORY" in df.columns and any(df["CATEGORY"].astype(str).str.lower().str.contains("other", na=False)):
            comments.append("Contains generic 'Other' categories — may need tuning.")
        if accuracy >= 0.9:
            comments.append("Excellent labeling quality.")
        elif accuracy >= 0.8:
            comments.append("Acceptable, but can be improved.")
        else:
            comments.append("Poor labeling quality — fine-tuning recommended.")
        review = {"accuracy": accuracy, "comments": " ".join(comments)}
        log_action("Reviewer Agent", "Review completed", {"accuracy": accuracy, "missing": int(missing)})
        return review
    except Exception as e:
        log_action("Reviewer Agent", "Error during review", {"error": str(e)})
        return {"accuracy": 0.0, "comments": f"Error during review: {e}"}

reviewer_agent = Agent(role="Reviewer Agent", goal="Review labeled data", backstory="Reviewer", llm=None)
print("Reviewer agent ready.")


Reviewer agent ready.


## Model loader (LLM)
We load a **small CPU-friendly model** (`google/gemma-2b-it`) to provide LLM fallback for labeling.
If you are on Colab with limited RAM, this is recommended.


In [ ]:
# core/model_loader.py
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

# Default: Gemma instruction-tuned model
MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemma-2b-it")

_tokenizer = None
_model = None


def load_model():
    """
    Loads Gemma model with 4-bit quantization if GPU exists,
    otherwise loads on CPU in float32.
    """
    global _tokenizer, _model

    if _model is not None:
        return _model, _tokenizer

    print(f"[model_loader] Loading Gemma model: {MODEL_NAME}")

    # ------------------------------------------
    # Check for GPU (CUDA / MPS / CPU fallback)
    # ------------------------------------------
    if torch.cuda.is_available():
        device = "cuda"
        print("[model_loader] Using CUDA GPU")

        quant = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        _model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",
            quantization_config=quant
        )

    elif torch.backends.mps.is_available():
        device = "mps"
        print("[model_loader] Using Apple MPS (M1/M2/M3)")

        _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        _model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16,
            device_map={"": device}
        )

    else:
        device = "cpu"
        print("[model_loader] Using CPU — this will be slower")

        _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        _model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float32,
            device_map={"": device}
        )

    return _model, _tokenizer


def generate_with_model(model, tokenizer, prompt, max_new_tokens=128):
    """
    Generates text using Gemma causal LM.
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


print("Gemma model loader is ready. Call load_model() to initialize the model.")


Gemma model loader is ready. Call load_model() to initialize the model.


## Labeling Agent (hybrid rule + LLM fallback)
This will:
- Use simple keyword rules for obvious categories
- Use the LLM fallback for unknown descriptions
- Save labeled CSV and JSON to datasets/labeled_data


In [ ]:
# agents/labeling_agent.py equivalent
import pandas as pd
from pathlib import Path

def detect_category(desc: str):
    if not isinstance(desc, str): return None
    d = desc.lower()
    if "atm" in d or "cash" in d: return "Cash Withdrawal"
    if "salary" in d and ("credit" in d or "salary" in d): return "Income"
    if "amazon" in d or "flipkart" in d or "myntra" in d: return "Shopping"
    if "interest" in d: return "Interest"
    return None

def label_with_llm(description: str):
    try:
        model, tokenizer = load_model()
        prompt = (
            "Classify the following bank transaction description into one of: "
            "[Income, Expense, Shopping, Interest, Transfer, Cash Withdrawal, Other].\n\n"
            f"Transaction: \"{description}\"\n\nRespond with the single category name."
        )
        out = generate_with_model(model, tokenizer, prompt, max_new_tokens=16)
        return out.split("\n")[0].strip()
    except Exception as e:
        print("[label_with_llm] LLM error:", e)
        return "Other"


def label_bank_statement(file_path: str):
    df = pd.read_csv(file_path)
    if "DESCRIPTION" not in df.columns:
        raise ValueError("CSV must have DESCRIPTION column.")
    categories = []
    for desc in df["DESCRIPTION"].astype(str):
        rule = detect_category(desc)
        if rule:
            categories.append(rule)
        else:
            categories.append(label_with_llm(desc))
    df["CATEGORY"] = categories
    out_csv = os.path.join("datasets/labeled_data", Path(file_path).name.replace(".csv","_labeled.csv"))
    df.to_csv(out_csv, index=False)
    # also save JSON
    out_json = out_csv.replace(".csv", ".json")
    df.to_json(out_json, orient="records", indent=2)
    log_action("Labeling Agent", "Labeled document", {"file": file_path, "output_csv": out_csv, "rows": len(df)})
    return out_csv

labeling_agent = Agent(role="Labeling Agent", goal="Label bank statements", backstory="Hybrid labeler", llm=None)
print("Labeling agent ready.")


Labeling agent ready.


## RAG: indexing and retrieval
We will use SentenceTransformers + FAISS to index labeled descriptions for retrieval.


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss, numpy as np
from pathlib import Path
EMB_MODEL = os.environ.get("EMBEDDING_MODEL", "all-MiniLM-L6-v2")
embed_model = SentenceTransformer(EMB_MODEL)

def index_labeled_file(labeled_csv: str):
    df = pd.read_csv(labeled_csv)
    texts = df["DESCRIPTION"].astype(str).tolist()
    if not texts:
        return None
    vectors = embed_model.encode(texts, show_progress_bar=False)
    vecs = np.array(vectors).astype("float32")
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(vecs)
    index_path = "outputs/rag_index.faiss"
    faiss.write_index(index, index_path)
    # store meta rows to sqlite
    conn = get_connection()
    cur = conn.cursor()
    for i, row in df.iterrows():
        meta = {"filename": labeled_csv, "row": int(i), "desc": row["DESCRIPTION"], "category": row.get("CATEGORY")}
        cur.execute("INSERT INTO meta (doc_id, filename, metadata, created_at) VALUES (?, ?, ?, ?)",
                    (f"{Path(labeled_csv).stem}_{i}", Path(labeled_csv).name, json.dumps(meta), datetime.utcnow().isoformat() + "Z"))
    conn.commit()
    log_action("RAG", "Indexed labeled file", {"file": labeled_csv, "count": len(texts)})
    return index_path

def retrieve_similar(query: str, top_k=3):
    index_path = "outputs/rag_index.faiss"
    if not os.path.exists(index_path):
        return []
    index = faiss.read_index(index_path)
    qvec = embed_model.encode([query]).astype("float32")
    D, I = index.search(qvec, top_k)
    # fetch top rows from meta table (simple approach)
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT metadata FROM meta ORDER BY id DESC LIMIT ?", (top_k*5,))
    rows = cur.fetchall()
    results = []
    for r in rows[:top_k]:
        try:
            results.append(json.loads(r["metadata"]))
        except:
            pass
    return results

print("RAG functions ready.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RAG functions ready.


## Q&A generator
Generate Q&A pairs from labeled data and append to a combined JSON + store in SQLite.


In [ ]:
from pathlib import Path

def generate_qa_from_labeled_data(labeled_csv: str):
    df = pd.read_csv(labeled_csv)
    qa_pairs = []
    for _, row in df.iterrows():
        desc = row.get("DESCRIPTION", "")
        cat = row.get("CATEGORY", "")
        if not desc or not cat:
            continue
        q = f"What type of transaction is '{desc}'?"
        a = f"This transaction is categorized as '{cat}'."
        qa_pairs.append({"question": q, "answer": a, "source_file": Path(labeled_csv).name})
    combined_path = os.environ.get("QA_COMBINED_PATH", "datasets/qa_data/combined_qa.json")
    existing = []
    if os.path.exists(combined_path):
        try:
            existing = json.load(open(combined_path))
        except Exception:
            existing = []
    existing.extend(qa_pairs)
    # deduplicate
    unique = [dict(t) for t in {tuple(d.items()) for d in existing}]
    with open(combined_path, "w") as f:
        json.dump(unique, f, indent=2)
    # persist to sqlite
    conn = get_connection()
    cur = conn.cursor()
    for p in qa_pairs:
        cur.execute("INSERT INTO qa (question, answer, source_file, created_at) VALUES (?, ?, ?, ?)",
                    (p["question"], p["answer"], p["source_file"], datetime.utcnow().isoformat() + "Z"))
    conn.commit()
    log_action("QAGenerator", "Appended QA", {"added": len(qa_pairs), "total": len(unique)})
    return combined_path

print("QA generator ready.")


QA generator ready.


## Fine-tuner (local LoRA/PEFT stub)
This cell contains a CPU-friendly stub. For a real LoRA/PEFT training run, use `peft.get_peft_model` and Trainer (careful with resource usage).


In [ ]:
# ============================================================
# REAL FINE-TUNING IMPLEMENTATION (LoRA on flan-t5-base)
# ============================================================

# ============================================================
# REAL FINE-TUNING IMPLEMENTATION (LoRA on Google Gemma)
# ============================================================

import os
import json
from datasets import Dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel

# Make sure paths exist
os.makedirs("models/fine_tuned", exist_ok=True)
os.makedirs("datasets/qa_data", exist_ok=True)

FINETUNE_DIR = "models/fine_tuned"
MODEL_NAME = "google/gemma-3-1b-it"  # Change to desired Gemma variant
QA_DIR = "datasets/qa_data"


# ---------------------------------------------------------
# LOAD TRAINING DATA
# ---------------------------------------------------------
def load_training_data():
    data = []

    if os.path.exists(QA_DIR):
        for f in os.listdir(QA_DIR):
            if f.endswith(".json"):
                qa_list = json.load(open(os.path.join(QA_DIR, f)))
                for qa in qa_list:
                    if "question" in qa and "answer" in qa:
                        data.append({
                            "input_text": qa["question"],
                            "target_text": qa["answer"]
                        })

    if not data:
        print("❌ No Q&A data found for finetuning!")
        return None

    print(f"📌 Loaded {len(data)} samples for REAL finetuning.")
    return Dataset.from_list(data)


# ---------------------------------------------------------
# REAL LORA FINETUNING FUNCTION (Gemma)
# ---------------------------------------------------------
def fine_tune_local_model(mode="auto"):
    print(f"\n🚀 Starting REAL fine-tuning — mode = {mode}")

    dataset = load_training_data()
    if not dataset:
        return None

    # -------------------------------
    # Tokenizer & Model
    # -------------------------------
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Optional 4-bit quantization (QLoRA)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )

    # -------------------------------
    # LoRA Config
    # -------------------------------
    lora_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules="all-linear",  # target linear layers in Gemma
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)

    # -------------------------------
    # Preprocess function
    # -------------------------------
    def preprocess(examples):
        inputs = tokenizer(examples["input_text"], truncation=True, padding=True, max_length=256)
        targets = tokenizer(examples["target_text"], truncation=True, padding=True, max_length=128)

        # Causal LM: concatenate input + target
        input_ids = []
        labels = []
        for inp, tgt in zip(inputs["input_ids"], targets["input_ids"]):
            combined = inp + tgt
            input_ids.append(combined)
            labels.append(combined)

        # Pad sequences
        max_len = max(len(l) for l in labels)
        input_ids = [ids + [tokenizer.pad_token_id]*(max_len - len(ids)) for ids in input_ids]
        labels = [l + [tokenizer.pad_token_id]*(max_len - len(l)) for l in labels]

        return {
            "input_ids": input_ids,
            "attention_mask": [[1]*len(ids) + [0]*(max_len - len(ids)) for ids in input_ids],
            "labels": labels
        }

    tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

    # -------------------------------
    # Training Arguments
    # -------------------------------
    training_args = TrainingArguments(
        output_dir=FINETUNE_DIR,
        num_train_epochs=1,
        per_device_train_batch_size=1,
        learning_rate=5e-5,
        logging_dir=f"{FINETUNE_DIR}/logs",
        save_strategy="epoch",
        bf16=True,
        remove_unused_columns=False,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset
    )

    print("⏳ Training started… this may take time on CPU/GPU…")
    trainer.train()

    # -------------------------------
    # Save LoRA / Tokenizer
    # -------------------------------
    model.save_pretrained(FINETUNE_DIR)
    tokenizer.save_pretrained(FINETUNE_DIR)

    # -------------------------------
    # Optional: merge adapter into base model
    # -------------------------------
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    peft_model = PeftModel.from_pretrained(base_model, FINETUNE_DIR)
    merged_model = peft_model.merge_and_unload()
    merged_model.save_pretrained(os.path.join(FINETUNE_DIR, "merged"))
    tokenizer.save_pretrained(os.path.join(FINETUNE_DIR, "merged"))

    print(f"✅ FINETUNING COMPLETE! Saved to: {FINETUNE_DIR}")
    return FINETUNE_DIR




## Crew setup / main runner
This cell ties everything together: Planner -> Executor -> Label -> Index -> QA -> Review -> conditional fine-tune.


In [ ]:
def run_audit_query(file_path: str):
    log_action("Crew", "Start pipeline", {"file": file_path})
    plan = plan_task(f"Analyze and label: {file_path}")
    labeled_csv = label_bank_statement(file_path)  # labels + creates JSON
    index_path = index_labeled_file(labeled_csv)
    qa_path = generate_qa_from_labeled_data(labeled_csv)
    review = simple_review_check(labeled_csv)
    log_action("Crew", "Completed pipeline", {"labeled": labeled_csv, "qa": qa_path, "review": review})
    # Decide fine-tuning requirement
    need_finetune = review.get("accuracy", 0) < 0.85
    return {"labeled_file": labeled_csv, "qa_file": qa_path, "review": review, "need_finetune": need_finetune}

print("Main runner ready.")


Main runner ready.


## Create a sample unlabelled CSV (small) to test the pipeline
This cell writes a small CSV to `datasets/sample_bank.csv`. You can replace it with your own CSV file later.


In [ ]:
import pandas as pd
sample = [
    {"DATE": "2025-01-10", "DESCRIPTION": "ATM Withdrawal", "DEBIT": 500, "CREDIT": "", "BALANCE": 4500},
    {"DATE": "2025-01-12", "DESCRIPTION": "Salary Credit", "DEBIT": "", "CREDIT": 2000, "BALANCE": 6500},
    {"DATE": "2025-01-15", "DESCRIPTION": "Amazon Purchase", "DEBIT": 300, "CREDIT": "", "BALANCE": 6200},
    {"DATE": "2025-01-17", "DESCRIPTION": "Salary credited", "DEBIT": 250, "CREDIT": "", "BALANCE": 5950},
    {"DATE": "2025-01-20", "DESCRIPTION": "Salary credited", "DEBIT": 1000, "CREDIT": "", "BALANCE": 4950}
]
df = pd.DataFrame(sample)
sample_path = "datasets/sample_bank.csv"
df.to_csv(sample_path, index=False)
print("Sample CSV written to", sample_path)


Sample CSV written to datasets/sample_bank.csv


## Optional: load LLM model now
Loading the LLM model downloads weights and takes time. If you want the hybrid LLM fallback active, run the cell below. Otherwise skip to running the pipeline and the rule-based labels will apply.


In [ ]:
!pip install -U bitsandbytes

In [ ]:
# Run only if you want to use LLM fallback now (may take time).


try:
    model, tokenizer = load_model()
    print("Model loaded.")
except Exception as e:
    print("Model load failed:", e)


[model_loader] Loading Gemma model: google/gemma-3-1b-it
[model_loader] Using CUDA GPU


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Model loaded.


## If review indicates need for fine-tuning
If `need_finetune` is True, you can run the `fine_tune_local_model()` to perform fine-tuning (stub or real).



In [ ]:
result = run_audit_query("datasets/sample_bank.csv")
print("Pipeline result summary:")
print(result["review"])
print("Need finetune:", result["need_finetune"])
print("Labeled file:", result["labeled_file"])
print("QA combined path:", result["qa_file"])
# show preview of labeled data
import pandas as pd
display(pd.read_csv(result["labeled_file"]))


/tmp/ipython-input-2730710012.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat() + "Z"
/tmp/ipython-input-2730710012.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().isoformat() + "Z"
/tmp/ipython-input-4146585766.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  (f"{Path(labeled_csv).stem}_{i}", Path(labeled_csv).name, json.dumps(meta), datetime.utcnow().isoformat() + "Z"))
/tmp/ipython-input-2730710012.py:6: DeprecationWarning: datetime.datetime.ut

Pipeline result summary:
{'accuracy': np.float64(0.95), 'comments': 'Excellent labeling quality.'}
Need finetune: False
Labeled file: datasets/labeled_data/sample_bank_labeled.csv
QA combined path: datasets/qa_data/combined_qa.json


,DATE,DESCRIPTION,DEBIT,CREDIT,BALANCE,CATEGORY
0,2025-01-10,ATM Withdrawal,500.0,NaN,4500,Cash Withdrawal
1,2025-01-12,Salary Credit,NaN,2000.0,6500,Income
2,2025-01-15,Amazon Purchase,300.0,NaN,6200,Shopping
3,2025-01-17,Salary credited,250.0,NaN,5950,Income
4,2025-01-20,Salary credited,1000.0,NaN,4950,Income


In [ ]:
if result.get("need_finetune"):
    out = fine_tune_local_model(mode="auto", sample_limit=50)
    print("Fine-tuning output:", out)
else:
    print("No fine-tuning needed (review good).")


No fine-tuning needed (review good).


## global finetuning

In [ ]:
def global_fine_tune():
    print("\n🔥 Running GLOBAL fine-tuning on ALL collected Q&A data")
    return fine_tune_local_model(mode="global")

# Run manually:
# out = global_fine_tune()
# out


In [ ]:
# Run global fine-tuning manually
out = global_fine_tune()  # remove limit for all samples
print("Model stored at:", out)



🔥 Running GLOBAL fine-tuning on ALL collected Q&A data

🚀 Starting REAL fine-tuning — mode = global
📌 Loaded 4 samples for REAL finetuning.


`torch_dtype` is deprecated! Use `dtype` instead!


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

⏳ Training started… this may take time on CPU/GPU…


Step,Training Loss


✅ FINETUNING COMPLETE! Saved to: models/fine_tuned
Model stored at: models/fine_tuned


## creating streamlt application

In [ ]:

%%writefile app.py
import streamlit as st
import pandas as pd
import json
import os
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, PeftModel
from huggingface_hub import login, whoami

# ===========================================================
#                HUGGING FACE LOGIN (for private Gemma)
# ===========================================================
HF_TOKEN = os.environ.get("HF_TOKEN", None)
if HF_TOKEN is None:
    st.sidebar.info("Log in to Hugging Face to access gated models")
    HF_TOKEN = st.sidebar.text_input("Paste Hugging Face token", type="password")
    if HF_TOKEN:
        login(HF_TOKEN)
user_info = whoami() if HF_TOKEN else None

# ===========================================================
#                   DATABASE (SQLite)
# ===========================================================
DB_PATH = "audit_logs.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            agent TEXT,
            action TEXT,
            details TEXT
        );
    """)
    conn.commit()
    conn.close()

def log_action(agent, action, details):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO logs (timestamp, agent, action, details)
        VALUES (?, ?, ?, ?)
    """, (datetime.now().isoformat(), agent, action, json.dumps(details)))
    conn.commit()
    conn.close()

init_db()

# ===========================================================
#                   AGENTS (CREW-LIKE)
# ===========================================================
def plan_task(file_path):
    steps = [
        "Load CSV",
        "Hybrid Label transactions",
        "Generate Q&A",
        "Review accuracy",
        "Fine-tune if needed"
    ]
    log_action("Planner Agent", "Plan created", {"steps": steps})
    return steps

# -------------------------------
# Rule-based labeling
# -------------------------------
def rule_based_label(desc):
    d = desc.lower()
    if "salary" in d:
        return "Income"
    if "atm" in d or "cash" in d:
        return "Cash Withdrawal"
    if "amazon" in d or "flipkart" in d:
        return "Online Shopping"
    if "interest" in d:
        return "Interest Earned"
    return "Other"

# -------------------------------
# Hybrid labeling using Gemma
# -------------------------------
MODEL_NAME = "google/gemma-3-1b-it"
_tokenizer = None
_model = None

def load_gemma_model():
    global _tokenizer, _model
    if _model is not None:
        return _model, _tokenizer

    bnb_config = None
    if torch.cuda.is_available():
        try:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_quant_type="nf4"
            )
        except ImportError:
            st.warning("bitsandbytes not installed, using float16")
            bnb_config = None

    _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=HF_TOKEN)
    _model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_config,
        use_auth_token=HF_TOKEN,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
    )
    return _model, _tokenizer

def gemma_label(desc):
    model, tokenizer = load_gemma_model()
    # categories = ["Income", "Expense", "Shopping", "Interest", "Transfer", "Cash Withdrawal", "Other"]
    prompt = f"Respond only 'Other payments'"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=16)
    label = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print("gemma predicted labels are, :",label)
    return label.strip()

def hybrid_label(desc):
    label = rule_based_label(desc)
    print(label)

    if label != "Other":
        return label
    try:
        return gemma_label(desc)
    except Exception:
        return "Other"

def label_csv(file_path):
    df = pd.read_csv(file_path)
    df["CATEGORY"] = df["DESCRIPTION"].apply(hybrid_label)

    os.makedirs("datasets/labeled_data", exist_ok=True)
    out_csv = "datasets/labeled_data/labeled_output.csv"
    df.to_csv(out_csv, index=False)

    log_action("Labeling Agent", "CSV labeled", {"rows": len(df)})
    return out_csv

# -------------------------------
# Q&A Generator
# -------------------------------
def generate_qa(labeled_csv):
    df = pd.read_csv(labeled_csv)
    qa_pairs = []
    for _, row in df.iterrows():
        q = f"What type of transaction is '{row['DESCRIPTION']}'?"
        a = f"It is categorized as '{row['CATEGORY']}'."
        qa_pairs.append({"question": q, "answer": a})
    os.makedirs("datasets/qa_data", exist_ok=True)
    qa_path = "datasets/qa_data/qa_output.json"
    json.dump(qa_pairs, open(qa_path, "w"), indent=2)
    log_action("Q&A Agent", "Generated Q&A", {"count": len(qa_pairs)})
    return qa_path

# -------------------------------
# Reviewer Agent
# -------------------------------
def review_labeled_output(labeled_csv):
    df = pd.read_csv(labeled_csv)
    accuracy = round(0.75 + (len(df) % 20)/100, 2)
    suggest_ft = accuracy < 0.85
    result = {"accuracy": accuracy, "suggest_finetune": suggest_ft, "comments": "Hybrid labeling review."}
    log_action("Reviewer Agent", "Review done", result)
    return result

# -------------------------------
# Fine-tuning Agent
# -------------------------------
FINETUNE_DIR = "models/fine_tuned"

def load_training_data():
    qa_path = "datasets/qa_data/qa_output.json"
    if not os.path.exists(qa_path):
        return None
    df = pd.read_json(qa_path)
    return Dataset.from_pandas(df)

def fine_tune_model():
    dataset = load_training_data()
    if dataset is None:
        return None

    bnb_config = None
    if torch.cuda.is_available():
        try:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_quant_type="nf4"
            )
        except ImportError:
            st.warning("bitsandbytes not installed, using float16")
            bnb_config = None

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=HF_TOKEN)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_config,
        use_auth_token=HF_TOKEN,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
    )

    lora_cfg = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules="all-linear",
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_cfg)

    def preprocess(batch):
        input_ids, labels = [], []
        for q, a in zip(batch["question"], batch["answer"]):
            combined = tokenizer.encode(q + " " + a, truncation=True, max_length=256)
            input_ids.append(combined)
            labels.append(combined)
        max_len = max(len(l) for l in labels)
        input_ids = [ids + [tokenizer.pad_token_id]*(max_len-len(ids)) for ids in input_ids]
        labels = [l + [tokenizer.pad_token_id]*(max_len-len(l)) for l in labels]
        attention_mask = [[1]*len(ids) + [0]*(max_len-len(ids)) for ids in input_ids]
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

    tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

    args = TrainingArguments(
        output_dir=FINETUNE_DIR,
        num_train_epochs=1,
        per_device_train_batch_size=1,
        logging_steps=5,
        save_strategy="epoch",
        bf16=torch.cuda.is_available(),
        report_to="none"
    )

    trainer = Trainer(model=model, args=args, train_dataset=tokenized_dataset)
    trainer.train()

    model.save_pretrained(FINETUNE_DIR)
    tokenizer.save_pretrained(FINETUNE_DIR)

    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, use_auth_token=HF_TOKEN,
                                                      device_map="auto", torch_dtype=torch.bfloat16)
    peft_model = PeftModel.from_pretrained(base_model, FINETUNE_DIR)
    merged_model = peft_model.merge_and_unload()
    merged_model.save_pretrained(os.path.join(FINETUNE_DIR, "merged"))
    tokenizer.save_pretrained(os.path.join(FINETUNE_DIR, "merged"))

    log_action("FineTuner Agent", "Fine-tuning complete", {"path": FINETUNE_DIR})
    return FINETUNE_DIR

# ===========================================================
#                STREAMLIT UI STARTS HERE
# ===========================================================
st.set_page_config(page_title="Audit AI (Gemma Hybrid)", layout="wide")
st.title("💼 AI-Powered Bank Statement Auditor (Gemma + Hybrid)")

st.sidebar.header("⚙ Controls")
uploaded_file = st.sidebar.file_uploader("Upload bank CSV", type=["csv"])

if st.sidebar.button("Run Analysis"):
    if not uploaded_file:
        st.error("Upload a CSV first!")
    else:
        os.makedirs("datasets", exist_ok=True)
        csv_path = f"datasets/{uploaded_file.name}"
        with open(csv_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        st.success("File uploaded!")

        plan = plan_task(csv_path)
        labeled_csv = label_csv(csv_path)
        df = pd.read_csv(labeled_csv)
        st.subheader("📌 Labeled Data Preview")
        st.dataframe(df.head())

        st.subheader("📊 Spending Analytics")
        cat_counts = df["CATEGORY"].value_counts()
        fig, ax = plt.subplots()
        ax.pie(cat_counts, labels=cat_counts.index, autopct="%1.1f%%")
        st.pyplot(fig)

        qa_path = generate_qa(labeled_csv)
        st.subheader("💬 Sample Q&A")
        st.json(json.load(open(qa_path))[:5])

        st.subheader("🔍 Review Score")
        review = review_labeled_output(labeled_csv)
        st.json(review)

        if review["suggest_finetune"]:
            st.warning("⚠ Low accuracy — Fine-tuning recommended.")
            if st.button("Run Fine-Tuning Now"):
                with st.spinner("Fine-tuning model…"):
                    out = fine_tune_model()
                st.success(f"Model fine-tuned → {out}")

if st.sidebar.button("Global Fine-Tuning"):
    with st.spinner("Training on all Q&A files…"):
        out = fine_tune_model()
    if out:
        st.success(f"Model saved to {out}")
    else:
        st.error("No Q&A data found!")

if st.sidebar.button("Show Logs"):
    conn = sqlite3.connect(DB_PATH)
    df_logs = pd.read_sql("SELECT * FROM logs ORDER BY id DESC LIMIT 20", conn)
    conn.close()
    st.subheader("🧾 Recent Logs")
    st.dataframe(df_logs)



Writing app.py


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken <your_ngrok_token>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:



from pyngrok import ngrok

public_url = ngrok.connect("8050", proto="http")
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://7bf1f347f243.ngrok-free.app" -> "http://localhost:8050"


In [ ]:
!streamlit run --server.port 8050 app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8050
  Network URL: http://172.28.0.12:8050
  External URL: http://136.110.32.43:8050

2025-11-22 11:46:31.021633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763811991.101213    9967 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763811991.136269    9967 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763811991.217352    9967 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763811991.217410    9967 computation_placer.cc:177] computation placer al